<a href="https://colab.research.google.com/github/shicong0720/Finding-Healthy-Community-Design-Criteria-with-AI/blob/main/2_7_ANN__combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import os
import numpy as np
import random
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive',force_remount=True)
os.chdir("/content/gdrive/My Drive/big data and city/")

Mounted at /content/gdrive


In [ ]:

import pandas as pd

data = pd.read_csv('combine_train.csv',header=None)
data = data.values.tolist()

maxi = [-99999999999 for i in range(len(data[0]))]
mini = [99999999999 for i in range(len(data[0]))]

for i in range(len(data)):
    for j in range(len(data[i])):
        if data[i][j]>maxi[j]:
            maxi[j] = data[i][j]
        if data[i][j]<mini[j]:
            mini[j] = data[i][j]

#maxi = [100000000000,200000000000,20]
#mini = [0,0,0]

for i in range(len(data)):
    for j in range(len(data[i])):
        data[i][j] = (data[i][j]-mini[j])/(maxi[j]-mini[j])


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

def initialize():
    #placeholder
    x = tf.placeholder(tf.float32, [None, D_in])
    y = tf.placeholder(tf.float32, [None, D_out])
    w = {
        'h1': tf.Variable(tf.random_normal([D_in, D_h1])),
        'h2': tf.Variable(tf.random_normal([D_h1, D_h2])),
        'h3': tf.Variable(tf.random_normal([D_h2, D_h3])),
        'h4': tf.Variable(tf.random_normal([D_h3, D_h4])),
        'h5': tf.Variable(tf.random_normal([D_h4, D_h5])),
        'h6': tf.Variable(tf.random_normal([D_h5, D_h6])),
        'h7': tf.Variable(tf.random_normal([D_h6, D_h7])),
        'h8': tf.Variable(tf.random_normal([D_h7, D_h8])),
        'h9': tf.Variable(tf.random_normal([D_h8, D_h9])),
        'h10': tf.Variable(tf.random_normal([D_h9, D_h10])),
        'out': tf.Variable(tf.random_normal([D_h10, D_out]))
    }
    b = {
        'h1': tf.Variable(tf.random_normal([D_h1])),
        'h2': tf.Variable(tf.random_normal([D_h2])),
        'h3': tf.Variable(tf.random_normal([D_h3])),
        'h4': tf.Variable(tf.random_normal([D_h4])),
        'h5': tf.Variable(tf.random_normal([D_h5])),
        'h6': tf.Variable(tf.random_normal([D_h6])),
        'h7': tf.Variable(tf.random_normal([D_h7])),
        'h8': tf.Variable(tf.random_normal([D_h8])),
        'h9': tf.Variable(tf.random_normal([D_h9])),
        'h10': tf.Variable(tf.random_normal([D_h10])),
        'out': tf.Variable(tf.random_normal([D_out]))
    }

    #activation functions #tf.layers.batch_normalization()
    def multilayer_perceptron(x):
        h1_layer = tf.sigmoid(tf.add(tf.matmul(x, w['h1']), b['h1']))
        h2_layer = tf.sigmoid(tf.add(tf.matmul(h1_layer, w['h2']), b['h2']))
        h3_layer = tf.sigmoid(tf.add(tf.matmul(h2_layer, w['h3']), b['h3']))
        h4_layer = tf.sigmoid(tf.add(tf.matmul(h3_layer, w['h4']), b['h4']))
        h5_layer = tf.sigmoid(tf.add(tf.matmul(h4_layer, w['h5']), b['h5']))
        h6_layer = tf.sigmoid(tf.add(tf.matmul(h5_layer, w['h6']), b['h6']))
        h7_layer = tf.sigmoid(tf.add(tf.matmul(h6_layer, w['h7']), b['h7']))
        h8_layer = tf.sigmoid(tf.add(tf.matmul(h7_layer, w['h8']), b['h8']))
        h9_layer = tf.sigmoid(tf.add(tf.matmul(h8_layer, w['h9']), b['h9']))
        h10_layer = tf.sigmoid(tf.add(tf.matmul(h9_layer, w['h10']), b['h10']))
        out_layer = tf.sigmoid(tf.add(tf.matmul(h10_layer, w['out']), b['out']))
        return out_layer
    pred = multilayer_perceptron(x)

    #loss function
    #cost = tf.reduce_sum(tf.where(tf.greater(pred, 0.932), 10*(y-pred)*(y-pred),
    #                     tf.where(tf.greater(0.042, pred), 10*(y-pred)*(y-pred),
    #                     (y-pred)*(y-pred))))
    cost = tf.reduce_sum((y-pred)*(y-pred))

    #optimizer and others
    optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
    init = tf.global_variables_initializer()
    variables_dict = {
        'b1': b['h1'],
        'b2': b['h2'],
        'b3': b['h3'],
        'b4': b['h4'],
        'b5': b['h5'],
        'b6': b['h6'],
        'b7': b['h7'],
        'b8': b['h8'],
        'b9': b['h9'],
        'b10': b['h10'],
        'bout': b['out'],
        'w1': w['h1'],
        'w2': w['h2'],
        'w3': w['h3'],
        'w4': w['h4'],
        'w5': w['h5'],
        'w6': w['h6'],
        'w7': w['h7'],
        'w8': w['h8'],
        'w9': w['h9'],
        'w10': w['h10'],
        'wout': w['out']
    }
    saver = tf.train.Saver(variables_dict)
    saver = tf.train.Saver(max_to_keep=1000, keep_checkpoint_every_n_hours=1)
    return x, y, pred, cost, optimizer, init, saver


def evaluate(filename, dimensionx):
    xdata = []
    with open(filename) as f:
        lines = f.readlines()
    temp = []
    for line in lines:
        temp.append(float(line))
        if len(temp)==dimensionx:
            xdata.append(temp)
            temp = []
    
    with tf.Session() as sess:
        saver.restore(sess, "./model.ckpt")
        yhat = pred.eval(feed_dict = {x: xdata})
        
        
def evaluatewhentrain(xdata, modelfile):
    with tf.Session() as sess:
        saver.restore(sess, modelfile)
        yhat = pred.eval(feed_dict = {x: xdata})
    return yhat


def train(X_test, y_test):
    with tf.Session() as sess:
        sess.run(init)
        merged_all = tf.summary.merge_all()
        writer = tf.summary.FileWriter('logs/', sess.graph)
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(num/batch_size)
            if num%batch_size != 0:
                total_batch = total_batch + 1
            for i in range(total_batch):
                batch_xs = xdata[i*batch_size:(i+1)*batch_size]
                batch_ys = ydata[i*batch_size:(i+1)*batch_size]
                _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs, y: batch_ys})
                avg_cost += c / total_batch
            if (epoch+1) % display_step == 0 or epoch == 0:
                print("Epoch:", '%04d' % (epoch+1), " cost=", "{:.9f}".format(avg_cost))
                #merged = sess.run(merged_all, feed_dict={x: batch_xs, y: batch_ys})
                #writer.add_summary(merged, epoch)
            if (epoch+1) % save_step == 0:
                saver.save(sess, "./model"+str(epoch)+".ckpt")
                print("save model done.")
                yhat = np.array(evaluatewhentrain(X_test,"./model"+str(epoch)+".ckpt"))
                accuracy = 100 - np.around(np.median(np.abs(yhat-y_test)/yhat*100,axis=0),decimals=1)
                print('5-layer ANN', accuracy, '% Step '+str(epoch))

In [ ]:
#training settings
training_epochs = 200000
save_step = 1000

batch_size = 100
display_step = 1000

#network structure
D_in, D_out = 14, 16
D_h1 = 14
D_h2 = 14
D_h3 = 14
D_h4 = 14
D_h5 = 14
D_h6 = 14
D_h7 = 14
D_h8 = 14
D_h9 = 14
D_h10 = 14

#initialize
x, y, pred, cost, optimizer, init, saver = initialize()

#load data
xdata = np.array(data)[:,0:14]
ydata = np.array(data)[:,14:]
X_test = np.array(data)[:,0:14]
y_test = np.array(data)[:,14:]

num = len(xdata)
print(xdata)
print(ydata)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  9.42827419e-04 2.86608123e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  2.00173451e-03 7.67276577e-05]
 [0.00000000e+00 0.00000000e+00 2.18483723e-04 ... 0.00000000e+00
  7.87908754e-02 1.98964572e-03]
 ...
 [5.97014925e-04 2.95159386e-03 1.74786978e-03 ... 0.00000000e+00
  2.41530466e-03 1.35842231e-03]
 [1.13432836e-02 2.00708383e-02 1.94450513e-02 ... 1.08674054e-02
  2.67860439e-02 7.90637644e-03]
 [0.00000000e+00 0.00000000e+00 2.18483723e-04 ... 0.00000000e+00
  6.76863764e-04 2.58131879e-04]]
[[0.03069054 0.39055794 0.43283582 ... 0.26732673 0.19512195 0.03492063]
 [0.40409207 0.33261803 0.5        ... 0.62376238 0.29268293 0.35555556]
 [0.12787724 0.35622318 0.52238806 ... 0.56930693 0.25609756 0.14603175]
 ...
 [0.0511509  0.33690987 0.5        ... 0.2970297  0.17073171 0.04761905]
 [0.04347826 0.36695279 0.27238806 ... 0.23762376 0.2195122  0.04126984]
 [0.59079284 0.27682403 0.4291

In [ ]:
train(X_test, y_test)

Epoch: 0001  cost= 254.471441905
Epoch: 1000  cost= 27.063699245
save model done.
INFO:tensorflow:Restoring parameters from ./model999.ckpt
5-layer ANN [53.1 81.8 89.9 77.7 83.9 91.8 82.3 83.7 69.7 84.7 77.6 75.8 76.8 78.8
 82.4 66.1] % Step 999
Epoch: 2000  cost= 26.518977324
save model done.
INFO:tensorflow:Restoring parameters from ./model1999.ckpt
5-layer ANN [53.9 82.3 89.9 77.9 84.3 91.9 82.5 83.9 70.1 85.2 77.4 76.2 76.6 78.9
 81.9 67.4] % Step 1999
Epoch: 3000  cost= 26.092969577
save model done.
INFO:tensorflow:Restoring parameters from ./model2999.ckpt
5-layer ANN [55.4 82.7 90.1 78.1 84.6 91.9 83.6 84.2 70.4 85.4 77.7 76.4 77.1 79.1
 82.9 67.3] % Step 2999
Epoch: 4000  cost= 25.479581833
save model done.
INFO:tensorflow:Restoring parameters from ./model3999.ckpt
5-layer ANN [56.8 83.9 90.1 79.5 84.8 92.1 84.6 84.7 70.6 85.3 77.9 76.7 77.2 79.3
 83.3 67.3] % Step 3999
Epoch: 5000  cost= 23.628208160
save model done.
INFO:tensorflow:Restoring parameters from ./model4999.ckpt
5

In [ ]:
test = []
with open("combine_test.csv", 'r') as t:
    reader = csv.reader(t)
    for row in reader:
        temp = np.array(row).astype(np.float).tolist()
        test.append(temp)

for i in range(len(test)):
    for j in range(len(test[i])):
        test[i][j] = (test[i][j]-mini[j])/(maxi[j]-mini[j])

X_test2 = np.array(test)[:,0:14]
y_test2 = np.array(test)[:,14:]

In [ ]:
X_pre = X_test2

with tf.Session() as sess:
    saver.restore(sess, "./model199999.ckpt")
    yhat = pred.eval(feed_dict = {x: X_pre})

print(yhat)

INFO:tensorflow:Restoring parameters from ./model199999.ckpt
[[0.60365576 0.286088   0.49202853 ... 0.67202353 0.2840007  0.47526515]
 [0.00807938 0.49494907 0.3897457  ... 0.08911502 0.22014895 0.01596546]
 [0.6832226  0.22655487 0.6124246  ... 0.91951394 0.23916149 0.679098  ]
 ...
 [0.13688192 0.3446743  0.58200616 ... 0.394466   0.21511808 0.16795221]
 [0.27698588 0.21663237 0.6733145  ... 0.562166   0.1461826  0.22245976]
 [0.18141013 0.48600087 0.47938642 ... 0.4832477  0.36746    0.28565556]]


In [ ]:
import pandas as pd
delta = pd.DataFrame((yhat-y_test2)/y_test2)
delta
delta.mean()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


0          inf
1     0.300053
2     0.042074
3     0.321586
4     0.186260
5    -0.009969
6          inf
7          inf
8     0.362570
9          inf
10    0.382016
11    0.189583
12    0.207093
13         inf
14         inf
15    0.742348
dtype: float64

In [ ]:
delta.std()

0          NaN
1     3.903602
2     0.323458
3     3.501048
4     1.173051
5     0.205137
6          NaN
7          NaN
8     1.313572
9          NaN
10    2.669274
11    0.796127
12    0.895008
13         NaN
14         NaN
15    4.115481
dtype: float64

In [ ]:
reader = tf.train.NewCheckpointReader('model199999.ckpt')
all_variables = reader.get_variable_to_shape_map()
print(all_variables)
w = reader.get_tensor("Variable")
print(type(w))
print(w.shape)
print(w)

{'beta2_power': [], 'beta1_power_3': [], 'beta1_power_2': [], 'beta1_power': [], 'Variable_9/Adam_1': [11, 11], 'Variable_9/Adam': [11, 11], 'Variable_87/Adam': [16], 'Variable_87': [16], 'Variable_86/Adam_1': [14], 'Variable_86/Adam': [14], 'Variable_84/Adam': [14], 'Variable_84': [14], 'Variable_82/Adam': [14], 'Variable_81/Adam_1': [14], 'Variable_81': [14], 'Variable_80/Adam': [14], 'Variable_80': [14], 'Variable_8/Adam_1': [11, 11], 'Variable_8/Adam': [11, 11], 'Variable_8': [11, 11], 'Variable_85/Adam_1': [14], 'Variable_79/Adam_1': [14], 'Variable_78/Adam_1': [14], 'Variable_78/Adam': [14], 'Variable_78': [14], 'Variable_77/Adam_1': [14], 'Variable_77/Adam': [14], 'Variable_87/Adam_1': [16], 'Variable_77': [14], 'Variable_76/Adam_1': [14, 16], 'Variable_76/Adam': [14, 16], 'Variable_75/Adam_1': [14, 14], 'Variable_75/Adam': [14, 14], 'Variable_74/Adam': [14, 14], 'Variable_73/Adam_1': [14, 14], 'Variable_73/Adam': [14, 14], 'Variable_73': [14, 14], 'Variable_82/Adam_1': [14], 'V

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import datetime
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import sys
import random
%matplotlib inline


X_train = np.array(data)[:,0:14]
y_train = np.array(data)[:,14:]

X_test = np.array(test)[:,0:14]
y_test = np.array(test)[:,14:]


# import xgboost as xgb
from sklearn.model_selection import train_test_split


# Different types of regressors
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

def my_custom_loss_func(ground_truth, predictions):
    return np.average(np.around(np.median(100*np.abs(predictions-ground_truth),axis=0),decimals=1))

#outtemp = []
#exdata, exdatastr = exeva(0,[],'',10)

n_estimators = 300
regr = DecisionTreeRegressor()
regr.fit(X_train,y_train)
#outtemp.append(findDT(regr,exdata,exdatastr))
#evadata('data/nne.txt', D_in, regr)
preds = regr.predict(X_test)
print ('Median Accuracy for DecisionTreeRegressor',100-my_custom_loss_func(y_test,preds),'%')

regr = BaggingRegressor(DecisionTreeRegressor(), n_estimators=n_estimators,n_jobs=-1)
regr.fit(X_train,y_train)
#outtemp.append(findDT(regr,exdata,exdatastr))
#evadata('data/nne.txt', D_in, regr)
preds = regr.predict(X_test)
print ('Median Accuracy for BaggingRegressor', 100-my_custom_loss_func(y_test,preds),'%')

regr = RandomForestRegressor(n_estimators=n_estimators, max_depth=None,min_samples_split=2, random_state=0,n_jobs=-1)
regr.fit(X_train,y_train)
#outtemp.append(findDT(regr,exdata,exdatastr))
#evadata('data/nne.txt', D_in, regr)
preds = regr.predict(X_test)
print ('Median Accuracy for RandomForestRegressor', 100-my_custom_loss_func(y_test,preds),'%')

regr = ExtraTreesRegressor(n_estimators=n_estimators, max_depth=None,min_samples_split=2, random_state=0,n_jobs=-1)
regr.fit(X_train,y_train)
#outtemp.append(findDT(regr,exdata,exdatastr))
#evadata('data/nne.txt', D_in, regr)
preds = regr.predict(X_test)
print ('Median Accuracy for ExtraTreesRegressor', 100-my_custom_loss_func(y_test,preds),'%')

reg = LinearRegression()
reg.fit(X_train,y_train)
preds = reg.predict(X_test)
print ('Median Accuracy for LinearRegressor', 100-my_custom_loss_func(y_test,preds),'%')

print ('Median Accuracy for ANN', 100-my_custom_loss_func(y_test,yhat),'%')

Median Accuracy for DecisionTreeRegressor 89.875 %
Median Accuracy for BaggingRegressor 91.7125 %
Median Accuracy for RandomForestRegressor 91.65 %
Median Accuracy for ExtraTreesRegressor 91.4125 %
Median Accuracy for LinearRegressor 90.96875 %
Median Accuracy for ANN 88.8375 %
